## Make optical flow data (It will take up a lot of memory)

In [ ]:
import cv2
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def calculate_optical_flow(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        angle = angle * 180 / np.pi / 2

        hsv = np.zeros_like(prev_frame)
        hsv[..., 1] = 255
        hsv[..., 0] = angle.astype(np.uint8)
        hsv[..., 2] = magnitude.astype(np.uint8)
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        cv2.imwrite(f"{output_dir}/{frame_idx:04d}.png", rgb)
        frame_idx += 1
        prev_gray = gray

    cap.release()

def process_video(video_file, video_dir, output_dir):
    video_path = os.path.join(video_dir, video_file)
    if video_path.endswith('.mp4'):
        output_path = os.path.join(output_dir, os.path.splitext(video_file)[0])
        calculate_optical_flow(video_path, output_path)

# Directory containing videos
video_dir = 'test'
output_dir = 'test_of_images'

# Using a ThreadPoolExecutor to process videos in parallel
with ThreadPoolExecutor(max_workers=128) as executor:
    futures = {executor.submit(process_video, video_file, video_dir, output_dir): video_file for video_file in os.listdir(video_dir) if video_file.endswith('.mp4')}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
        future.result()  # This retrieves any exceptions and does error handling

# All done!


## Re-posting as videos can save space

In [ ]:
import cv2
import os
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def images_to_video(image_folder, output_video, fps):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort()  # Ensure images are in the correct order

    # Read the first image to get video dimensions
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # Define video codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # Add images to video
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    # Release the video
    video.release()

def convert_folder_to_video(folder, input_dir, output_dir, fps):
    folder_path = os.path.join(input_dir, folder)
    if os.path.isdir(folder_path):
        output_video_path = os.path.join(output_dir, f"{folder}.mp4")
        images_to_video(folder_path, output_video_path, fps)

def main(input_dir, output_dir, fps, max_workers=32):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    folders = [folder for folder in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, folder))]

    # Using a ThreadPoolExecutor to process videos in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(convert_folder_to_video, folders, [input_dir]*len(folders), [output_dir]*len(folders), [fps]*len(folders)), total=len(folders)))

# Input and output directories
input_dir = 'test_of_images'  # Replace with your input directory path
output_dir = 'test_of_video'  # Replace with your output directory path
fps = 30  # Assuming the frame rate is 30, adjust as necessary

main(input_dir, output_dir, fps)
